In [2]:
using POMDPs, POMDPToolbox
using Distributions
include("roadnet_MDP.jl")

rewards (generic function with 1 method)

In [3]:
include("network_library.jl")
g = original_roadnet()
(reward_states,reward_vals) = rewards(g)
mdp = roadnet(0.6,0.9,reward_vals,reward_states,g)

13-element Array{roadnetState,1}:
 roadnetState(1, false) 
 roadnetState(2, false) 
 roadnetState(3, false) 
 roadnetState(4, false) 
 roadnetState(5, false) 
 roadnetState(6, false) 
 roadnetState(7, false) 
 roadnetState(8, false) 
 roadnetState(9, false) 
 roadnetState(10, false)
 roadnetState(11, false)
 roadnetState(12, false)
 roadnetState(13, true) 

In [3]:
sim(mdp,roadnetState(1),max_steps=1) do s
    println("state is: $s")
    a_set = [:n1,:n2,:n3,:n4]
    a = a_set[rand(1:end)] # choose random action
    println("moving to $a")
    return a
end

state is: roadnetState(1, false)
moving to n3


POMDPToolbox.MDPHistory{roadnetState,Symbol}(roadnetState[roadnetState(1, false), roadnetState(1, false)], Symbol[:n3], [-1.0], 0.9, Nullable{Exception}(), Nullable{Any}())

In [4]:
using DiscreteValueIteration

solver = ValueIterationSolver(max_iterations=1000, belres=1e-3)
policy = ValueIterationPolicy(mdp)

solve(solver, mdp, policy, verbose=true)

[Iteration 1   ] residual:   1E+03 | iteration runtime:      0.538 ms, (  0.000538 s total)
[Iteration 2   ] residual:        900 | iteration runtime:      0.503 ms, (   0.00104 s total)
[Iteration 3   ] residual:        810 | iteration runtime:      0.939 ms, (   0.00198 s total)
[Iteration 4   ] residual:        729 | iteration runtime:      0.454 ms, (   0.00243 s total)
[Iteration 5   ] residual:        656 | iteration runtime:      0.474 ms, (   0.00291 s total)
[Iteration 6   ] residual:        590 | iteration runtime:      0.510 ms, (   0.00342 s total)
[Iteration 7   ] residual:        531 | iteration runtime:      0.431 ms, (   0.00385 s total)
[Iteration 8   ] residual:        478 | iteration runtime:      0.423 ms, (   0.00427 s total)
[Iteration 9   ] residual:        430 | iteration runtime:      0.430 ms, (    0.0047 s total)
[Iteration 10  ] residual:        387 | iteration runtime:      0.435 ms, (   0.00514 s total)
[Iteration 11  ] residual:        349 | iteration run

[Iteration 90  ] residual:     0.0846 | iteration runtime:      0.658 ms, (    0.0617 s total)
[Iteration 91  ] residual:     0.0762 | iteration runtime:      0.860 ms, (    0.0626 s total)
[Iteration 92  ] residual:     0.0686 | iteration runtime:      0.826 ms, (    0.0634 s total)
[Iteration 93  ] residual:     0.0617 | iteration runtime:      0.718 ms, (    0.0641 s total)
[Iteration 94  ] residual:     0.0555 | iteration runtime:      1.026 ms, (    0.0651 s total)
[Iteration 95  ] residual:       0.05 | iteration runtime:      1.111 ms, (    0.0663 s total)
[Iteration 96  ] residual:      0.045 | iteration runtime:      1.219 ms, (    0.0675 s total)
[Iteration 97  ] residual:     0.0405 | iteration runtime:      5.539 ms, (     0.073 s total)
[Iteration 98  ] residual:     0.0364 | iteration runtime:      0.468 ms, (    0.0735 s total)
[Iteration 99  ] residual:     0.0328 | iteration runtime:      0.466 ms, (    0.0739 s total)
[Iteration 100 ] residual:     0.0295 | iteration 

DiscreteValueIteration.ValueIterationPolicy([513.103 543.471 488.124 488.124; 487.041 468.174 503.743 452.368; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], [543.471, 503.743, 491.062, 575.397, 586.094, 703.613, 708.35, 873.358, 589.864, 672.451  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [2, 3, 2, 3, 2, 3, 3, 4, 3, 3  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Symbol[:n1, :n2, :n3, :n4], true, roadnet(0.6, 0.9, [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1000.0], roadnetState[roadnetState(1, false), roadnetState(2, false), roadnetState(3, false), roadnetState(4, false), roadnetState(5, false), roadnetState(6, false), roadnetState(7, false), roadnetState(8, false), roadnetState(9, false), roadnetState(10, false), roadnetState(11, false), roadnetState(12, false), roadnetState(13, true)], {13, 19} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)))

In [6]:
for x in vertices(mdp.road_net)
    s = roadnetState(x)
    #display("node $x")
    #display(neighbors(mdp.road_net,s.node)')
    a = action(policy,s)
    display(a)
end

:n2

:n3

:n2

:n3

:n2

...

:n3

:n3

:n3

:n4

:n3

In [7]:
s = roadnetState(4)
hist = HistoryRecorder(max_steps=1000)

hist = simulate(hist,mdp,policy,s)

println("Total discounted reward: $(discounted_reward(hist))")

Total discounted reward: 526.7554100000001


In [8]:
for (s, a, sp) in eachstep(hist, "s,a,sp")
    @printf("s: %-26s  a: %-6s  s': %-26s\n", s, a, sp)
end

s: roadnetState(4, false)      a: n3      s': roadnetState(5, false)    
s: roadnetState(5, false)      a: n2      s': roadnetState(4, false)    
s: roadnetState(4, false)      a: n3      s': roadnetState(7, false)    
s: roadnetState(7, false)      a: n3      s': roadnetState(8, false)    
s: roadnetState(8, false)      a: n4      s': roadnetState(6, false)    
s: roadnetState(6, false)      a: n3      s': roadnetState(8, false)    
s: roadnetState(8, false)      a: n4      s': roadnetState(13, true)    
